In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, BatchNorm, global_add_pool, ChebConv, global_max_pool, SAGPooling, GATConv, GATv2Conv, TransformerConv, SuperGATConv, global_mean_pool, Linear
from torch.nn import BatchNorm1d
from math import floor
import torch
import torch_geometric
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from pathlib import Path

In [4]:
class EEGGraphConvNetLSTM(nn.Module):
    
    def __init__(self, reduced_sensors=True, sfreq=None, batch_size=256, **kwargs):
        super(EEGGraphConvNetLSTM, self).__init__()
        # Define and initialize hyperparameters
        self.sfreq = sfreq
        self.batch_size = batch_size
        #self.input_size = 8 if reduced_sensors else 62
        
        # Layers definition
        input_size = kwargs.pop("input_size", 1280)
        hidden_dim = 320
        
        # Graph convolutional layers
        self.conv1 = GCNConv(input_size, 640, cached=True, normalize=False)
        self.conv2 = GCNConv(640, 512, cached=True, normalize=False)
        self.conv3 = GCNConv(512, 256, cached=True, normalize=False)
        
        # Batch normalization
        self.batch_norm1 = BatchNorm(640, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm2 = BatchNorm(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm3 = BatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.batch_norm4 = BatchNorm(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        self.lstm = nn.LSTM(256, 256, 1, dropout=0.2)
        # Fully connected layers
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 2)
        
        # Xavier initializacion for fully connected layers
        self.fc1.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc2.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        self.fc3.apply(lambda x: nn.init.xavier_normal_(x.weight, gain=1) if isinstance(x, nn.Linear) else None)
        
        
    def forward(self, x, edge_index, edge_weigth, batch):
        
        x = F.leaky_relu(self.batch_norm1(self.conv1(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm2(self.conv2(x, edge_index, edge_weigth)), negative_slope=0.01)
        x = F.leaky_relu(self.batch_norm3(self.conv3(x, edge_index, edge_weigth)), negative_slope=0.01)
        x, _ = self.lstm(x)
        #x = F.leaky_relu(self.batch_norm4(self.conv4(x, edge_index, edge_weigth)), negative_slope=0.01)
        #x = F.dropout(batch_norm_out, p=0.2, training=self.training)
        
        # Global add pooling
        add_pool = global_add_pool(x, batch=batch)
        # Apply fully connected layters
        out = F.leaky_relu(self.fc1(add_pool), negative_slope=0.01)
        out = F.dropout(out, p = 0.2, training=self.training)
        out = F.leaky_relu(self.fc2(out), negative_slope=0.01)
        #out = F.dropout(out, p = 0.2, training=self.training)
        out = F.leaky_relu(self.fc3(out))
        return out
        

In [5]:
def create_corrected_data_list(path):
    data_list = list()
    for file in path.iterdir():
        data_list.append(torch.load(file))
        
    corrected_data_list = list()
    
    for data in data_list:
    # print(data)
        data = torch_geometric.data.Data(
            x=torch.tensor(data.x),
            edge_index=torch.tensor(data.edge_index),
            edge_attr=torch.tensor(data.edge_attr),
            label=torch.tensor(data.label),
        )
        corrected_data_list.append(data)
        
    rm = [
      7,
      14+1,
      14+2,
      17+3,
      17+4,
      26+5,
      38+6,
      54+7,
      65+8,
      69+9
      ]

    dl = list()
    start = 0
    for r in rm:
        dl.extend(corrected_data_list[start:r])
        start = r + 1

    dl.extend(corrected_data_list[start:])
    dl_filterd = list()
    for data in dl:
        if data.label == 2:
            # print(data.label)
            # if data.label == 2:
            data.label = torch.tensor(1)
        dl_filterd.append(data)

    len(dl_filterd)
    return dl_filterd
            
    

# Raw Pearson

In [6]:
path = Path('graphs/raw_pearson/')


dl_filterd = create_corrected_data_list(path)

train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=64, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=32, shuffle=False, num_workers=0)
train_dl[0]

C:\Users\mashh\AppData\Local\Temp\ipykernel_34968\129136280.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(data.x),
C:\Users\mashh\AppData\Local\Temp\ipykernel_34968\129136280.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


Data(x=[19, 153550], edge_index=[2, 361], edge_attr=[19, 19], label=0)

In [8]:
model = EEGGraphConvNetLSTM(input_size = 153550)

c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [9]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()
print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 99303106


EEGGraphConvNetLSTM(
  (conv1): GCNConv(153550, 640)
  (conv2): GCNConv(640, 512)
  (conv3): GCNConv(512, 256)
  (batch_norm1): BatchNorm(640)
  (batch_norm2): BatchNorm(512)
  (batch_norm3): BatchNorm(256)
  (batch_norm4): BatchNorm(256)
  (lstm): LSTM(256, 256, dropout=0.2)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [10]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.518015	 acc=0.564103
Test  : epoch=1	 loss=2.052299	 acc=0.500000

Train : epoch=2	 loss=0.944445	 acc=0.653846
Test  : epoch=2	 loss=0.947090	 acc=0.562500

Train : epoch=3	 loss=0.362613	 acc=0.705128
Test  : epoch=3	 loss=0.655671	 acc=0.562500

Train : epoch=4	 loss=0.292389	 acc=0.717949
Test  : epoch=4	 loss=0.801168	 acc=0.562500

Train : epoch=5	 loss=0.301895	 acc=0.679487
Test  : epoch=5	 loss=1.620729	 acc=0.500000

Train : epoch=6	 loss=0.382083	 acc=0.717949
Test  : epoch=6	 loss=1.741892	 acc=0.500000

Train : epoch=7	 loss=0.269228	 acc=0.717949
Test  : epoch=7	 loss=0.896446	 acc=0.562500

Train : epoch=8	 loss=0.266489	 acc=0.846154
Test  : epoch=8	 loss=1.073627	 acc=0.562500

Train : epoch=9	 loss=0.228267	 acc=0.858974
Test  : epoch=9	 loss=2.436531	 acc=0.500000

Train : epoch=10	 loss=0.362411	 acc=0.782051
Test  : epoch=10	 loss=3.327008	 acc=0.500000



In [11]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## RAW PLI

In [12]:
path = Path('graphs/raw_pli/')


dl_filterd = create_corrected_data_list(path)

train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=64, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=32, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_34968\129136280.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x=torch.tensor(data.x),
C:\Users\mashh\AppData\Local\Temp\ipykernel_34968\129136280.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [13]:
model = EEGGraphConvNetLSTM(input_size = 153550)

c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()
print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 99303106


EEGGraphConvNetLSTM(
  (conv1): GCNConv(153550, 640)
  (conv2): GCNConv(640, 512)
  (conv3): GCNConv(512, 256)
  (batch_norm1): BatchNorm(640)
  (batch_norm2): BatchNorm(512)
  (batch_norm3): BatchNorm(256)
  (batch_norm4): BatchNorm(256)
  (lstm): LSTM(256, 256, dropout=0.2)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [15]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.944001	 acc=0.576923
Test  : epoch=1	 loss=3.475220	 acc=0.500000

Train : epoch=2	 loss=1.345836	 acc=0.641026
Test  : epoch=2	 loss=1.562918	 acc=0.562500

Train : epoch=3	 loss=0.454308	 acc=0.705128
Test  : epoch=3	 loss=1.333666	 acc=0.500000

Train : epoch=4	 loss=0.375936	 acc=0.692308
Test  : epoch=4	 loss=0.563343	 acc=0.562500

Train : epoch=5	 loss=0.244694	 acc=0.717949
Test  : epoch=5	 loss=0.802312	 acc=0.562500

Train : epoch=6	 loss=0.236556	 acc=0.730769
Test  : epoch=6	 loss=1.034955	 acc=0.500000

Train : epoch=7	 loss=0.235329	 acc=0.705128
Test  : epoch=7	 loss=0.887832	 acc=0.500000

Train : epoch=8	 loss=0.147699	 acc=0.833333
Test  : epoch=8	 loss=1.294311	 acc=0.562500

Train : epoch=9	 loss=0.129331	 acc=0.858974
Test  : epoch=9	 loss=1.036173	 acc=0.687500

Train : epoch=10	 loss=0.113313	 acc=0.948718
Test  : epoch=10	 loss=0.663588	 acc=0.875000



In [16]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.75      0.86         8
           1       0.80      1.00      0.89         8

    accuracy                           0.88        16
   macro avg       0.90      0.88      0.87        16
weighted avg       0.90      0.88      0.87        16



## Pearson Moments

In [17]:
path = Path('graphs/moments_pearson/')


dl_filterd = create_corrected_data_list(path)

train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=64, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=32, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_34968\129136280.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [18]:
model = EEGGraphConvNetLSTM(input_size = 6)

c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()
print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 1034946


EEGGraphConvNetLSTM(
  (conv1): GCNConv(6, 640)
  (conv2): GCNConv(640, 512)
  (conv3): GCNConv(512, 256)
  (batch_norm1): BatchNorm(640)
  (batch_norm2): BatchNorm(512)
  (batch_norm3): BatchNorm(256)
  (batch_norm4): BatchNorm(256)
  (lstm): LSTM(256, 256, dropout=0.2)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [20]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.846999	 acc=0.461538
Test  : epoch=1	 loss=0.695920	 acc=0.500000

Train : epoch=2	 loss=0.778846	 acc=0.692308
Test  : epoch=2	 loss=0.695376	 acc=0.500000

Train : epoch=3	 loss=0.538867	 acc=0.653846
Test  : epoch=3	 loss=3.931880	 acc=0.500000

Train : epoch=4	 loss=0.777267	 acc=0.653846
Test  : epoch=4	 loss=5.325059	 acc=0.500000

Train : epoch=5	 loss=0.906934	 acc=0.653846
Test  : epoch=5	 loss=4.414628	 acc=0.500000

Train : epoch=6	 loss=0.905335	 acc=0.653846
Test  : epoch=6	 loss=3.079530	 acc=0.500000

Train : epoch=7	 loss=0.737990	 acc=0.653846
Test  : epoch=7	 loss=2.072034	 acc=0.500000

Train : epoch=8	 loss=0.515325	 acc=0.653846
Test  : epoch=8	 loss=1.454544	 acc=0.500000

Train : epoch=9	 loss=0.419032	 acc=0.653846
Test  : epoch=9	 loss=1.134950	 acc=0.500000

Train : epoch=10	 loss=0.400340	 acc=0.653846
Test  : epoch=10	 loss=1.381717	 acc=0.500000



In [21]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## Moments PLI

In [22]:
path = Path('graphs/moments_pli/')


dl_filterd = create_corrected_data_list(path)

train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=64, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=32, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_34968\129136280.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [24]:
model = EEGGraphConvNetLSTM(input_size = 6)

In [25]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()
print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 1034946


EEGGraphConvNetLSTM(
  (conv1): GCNConv(6, 640)
  (conv2): GCNConv(640, 512)
  (conv3): GCNConv(512, 256)
  (batch_norm1): BatchNorm(640)
  (batch_norm2): BatchNorm(512)
  (batch_norm3): BatchNorm(256)
  (batch_norm4): BatchNorm(256)
  (lstm): LSTM(256, 256, dropout=0.2)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [26]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.707861	 acc=0.397436
Test  : epoch=1	 loss=0.696331	 acc=0.500000

Train : epoch=2	 loss=0.598268	 acc=0.653846
Test  : epoch=2	 loss=0.704921	 acc=0.500000

Train : epoch=3	 loss=0.809749	 acc=0.653846
Test  : epoch=3	 loss=0.736039	 acc=0.500000

Train : epoch=4	 loss=0.742485	 acc=0.653846
Test  : epoch=4	 loss=0.766159	 acc=0.500000

Train : epoch=5	 loss=0.516999	 acc=0.653846
Test  : epoch=5	 loss=0.905519	 acc=0.500000

Train : epoch=6	 loss=0.491121	 acc=0.653846
Test  : epoch=6	 loss=1.580807	 acc=0.500000

Train : epoch=7	 loss=0.627055	 acc=0.653846
Test  : epoch=7	 loss=1.896791	 acc=0.500000

Train : epoch=8	 loss=0.564187	 acc=0.653846
Test  : epoch=8	 loss=2.133830	 acc=0.500000

Train : epoch=9	 loss=0.646898	 acc=0.653846
Test  : epoch=9	 loss=2.197213	 acc=0.500000

Train : epoch=10	 loss=0.636284	 acc=0.653846
Test  : epoch=10	 loss=1.784198	 acc=0.500000



In [27]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## PSD CSD

In [28]:
path = Path('graphs/psd_csd/')


dl_filterd = create_corrected_data_list(path)

train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=64, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=32, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_34968\129136280.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [29]:
model = EEGGraphConvNetLSTM(input_size = 6)

c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [30]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()
print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 1034946


EEGGraphConvNetLSTM(
  (conv1): GCNConv(6, 640)
  (conv2): GCNConv(640, 512)
  (conv3): GCNConv(512, 256)
  (batch_norm1): BatchNorm(640)
  (batch_norm2): BatchNorm(512)
  (batch_norm3): BatchNorm(256)
  (batch_norm4): BatchNorm(256)
  (lstm): LSTM(256, 256, dropout=0.2)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [31]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.711846	 acc=0.525641
Test  : epoch=1	 loss=0.693191	 acc=0.500000

Train : epoch=2	 loss=0.672164	 acc=0.653846
Test  : epoch=2	 loss=0.713102	 acc=0.500000

Train : epoch=3	 loss=0.585177	 acc=0.653846
Test  : epoch=3	 loss=0.785440	 acc=0.500000

Train : epoch=4	 loss=0.527329	 acc=0.653846
Test  : epoch=4	 loss=0.859720	 acc=0.500000

Train : epoch=5	 loss=0.540956	 acc=0.653846
Test  : epoch=5	 loss=0.914882	 acc=0.500000

Train : epoch=6	 loss=0.523301	 acc=0.653846
Test  : epoch=6	 loss=0.963434	 acc=0.500000

Train : epoch=7	 loss=0.534079	 acc=0.653846
Test  : epoch=7	 loss=0.978721	 acc=0.500000

Train : epoch=8	 loss=0.518691	 acc=0.653846
Test  : epoch=8	 loss=0.977600	 acc=0.500000

Train : epoch=9	 loss=0.521367	 acc=0.653846
Test  : epoch=9	 loss=0.984440	 acc=0.500000

Train : epoch=10	 loss=0.523277	 acc=0.653846
Test  : epoch=10	 loss=0.986646	 acc=0.500000



In [32]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## PSD Pearson

In [33]:
path = Path('graphs/psd_pearson/')


dl_filterd = create_corrected_data_list(path)

train_dl, test_dl = train_test_split(dl_filterd, test_size=0.2, random_state=47744)
train_dataloader = torch_geometric.loader.DataLoader(dl_filterd, batch_size=64, shuffle=False, num_workers=0)
test_dataloader = torch_geometric.loader.DataLoader(test_dl, batch_size=32, shuffle=False, num_workers=0)

C:\Users\mashh\AppData\Local\Temp\ipykernel_34968\129136280.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index=torch.tensor(data.edge_index),


In [34]:
model = EEGGraphConvNetLSTM(input_size = 6)

c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [35]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.0001)
criterion = torch.nn.CrossEntropyLoss()
print(f"Model Params: {sum(p.numel() for p in model.parameters())}")
model.double()

Model Params: 1034946


EEGGraphConvNetLSTM(
  (conv1): GCNConv(6, 640)
  (conv2): GCNConv(640, 512)
  (conv3): GCNConv(512, 256)
  (batch_norm1): BatchNorm(640)
  (batch_norm2): BatchNorm(512)
  (batch_norm3): BatchNorm(256)
  (batch_norm4): BatchNorm(256)
  (lstm): LSTM(256, 256, dropout=0.2)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [36]:
epochs = 10
verbose = False

torch.autograd.set_detect_anomaly(False)

for epoch in range(1, epochs+1):
    model.train()

    losses, y_true, y_pred = list(), list(), list()
    

    
    count = 0
    for idx, data in enumerate(train_dataloader):
        # data = data[0]
        # print(f"{data=}")
        # print(f"{data.x.shape}")
        optimizer.zero_grad()
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out)
        # print(f"{out.shape=}")
        #print(out.shape)
        # print(data.label)

        loss = criterion(out, data.label)
        acc = (out.argmax(dim=1) == data.label).sum().item() / len(data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())
        # print(f'{count=}{out}')
        # print(out.argmax(dim=1), data.label, end=' ')
        if verbose: print(f"epoch={epoch}\t batch={idx+1} : loss={loss.item():.6f}\t acc={acc:.6f}", end='\n')
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.01)
        optimizer.step()
        # break
        count += 1
    print(f"Train : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n')

    model.eval()
    losses, y_true, y_pred = list(), list(), list()
    for idx, data in enumerate(test_dataloader):
        # data = data[0]
        # print(f"{data[0]=}")
        # print(f"{data.x.shape}")
    
        
        # out = model(x=data.x, edge_index=data.edge_index)
        out = model(x=data.x, edge_index=data.edge_index, edge_weigth=data.edge_attr, batch=data.batch)
        # print(out.edge_attr)
        # print(f"{out.shape=}")
        # print(out)

        loss = criterion(out, data.label)
        losses.append(loss.item())
        y_pred.extend(out.argmax(dim=1).tolist())
        y_true.extend(data.label.tolist())

    print(f"Test  : epoch={epoch}\t loss={sum(losses)/len(losses):.6f}\t acc={accuracy_score(y_true, y_pred):.6f}", end='\n\n')
    

Train : epoch=1	 loss=0.757262	 acc=0.487179
Test  : epoch=1	 loss=0.693176	 acc=0.500000

Train : epoch=2	 loss=0.515620	 acc=0.666667
Test  : epoch=2	 loss=1.730529	 acc=0.500000

Train : epoch=3	 loss=0.813548	 acc=0.653846
Test  : epoch=3	 loss=1.764509	 acc=0.500000

Train : epoch=4	 loss=0.720682	 acc=0.653846
Test  : epoch=4	 loss=1.806645	 acc=0.500000

Train : epoch=5	 loss=0.566253	 acc=0.653846
Test  : epoch=5	 loss=1.684644	 acc=0.500000

Train : epoch=6	 loss=0.411043	 acc=0.653846
Test  : epoch=6	 loss=1.673061	 acc=0.500000

Train : epoch=7	 loss=0.330216	 acc=0.653846
Test  : epoch=7	 loss=1.682278	 acc=0.500000

Train : epoch=8	 loss=0.390329	 acc=0.653846
Test  : epoch=8	 loss=1.622271	 acc=0.500000

Train : epoch=9	 loss=0.283018	 acc=0.653846
Test  : epoch=9	 loss=1.713523	 acc=0.500000

Train : epoch=10	 loss=0.278640	 acc=0.653846
Test  : epoch=10	 loss=2.141408	 acc=0.500000



In [37]:
from sklearn.metrics import confusion_matrix, classification_report

print(classification_report(y_true, y_pred))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         8
           1       0.50      1.00      0.67         8

    accuracy                           0.50        16
   macro avg       0.25      0.50      0.33        16
weighted avg       0.25      0.50      0.33        16



c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\mashh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,